<a href="https://colab.research.google.com/github/aqwvinh/Basic_guideline_for_Machine_Learning/blob/master/CareerCon_2019_(Classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importation

In [0]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn import metrics
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


In [24]:
# Import dataset from Kaggle
import os
os.environ['KAGGLE_USERNAME'] = "vinhnguyen94" # username from the json file
os.environ['KAGGLE_KEY'] = "eb32c9bc896313e2f65c5948036bda01" # key from the json file, create new one for each time
!kaggle competitions download -c career-con-2019 # api copied from kaggle

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
X_test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
y_train.csv: Skipping, found more recently modified local copy (use --force to force download)
X_train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
X_train = pd.read_csv("X_train.csv.zip")
X_test = pd.read_csv("X_test.csv.zip")
y_train = pd.read_csv("y_train.csv")

In [26]:
X_train.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410


In [27]:
X_test.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,0.91208,-0.38193,-0.050618,0.14028,-0.060205,0.071286,-0.18787,0.29492,2.8027,-9.6816
1,0_1,0,1,0.91220,-0.38165,-0.050573,0.14028,-0.033486,0.060210,-0.18206,0.14944,2.5408,-9.8521
2,0_2,0,2,0.91228,-0.38143,-0.050586,0.14032,-0.029686,0.029476,-0.18441,-0.49741,2.5853,-9.3835
3,0_3,0,3,0.91237,-0.38121,-0.050588,0.14035,-0.024217,0.037788,-0.18783,-0.32376,2.9966,-8.7415
4,0_4,0,4,0.91247,-0.38096,-0.050546,0.14042,-0.038047,0.083405,-0.20170,-0.70103,2.6498,-8.8432


In [28]:
y_train.head()

,series_id,group_id,surface
0,0,13,fine_concrete
1,1,31,concrete
2,2,20,concrete
3,3,31,concrete
4,4,22,soft_tiles


In [29]:
X_train["series_id"].nunique(), y_train.shape

(3810, (3810, 3))

### Missing values

In [30]:
# Check number of missing values
len(X_train) - X_train.count()

row_id                   0
series_id                0
measurement_number       0
orientation_X            0
orientation_Y            0
orientation_Z            0
orientation_W            0
angular_velocity_X       0
angular_velocity_Y       0
angular_velocity_Z       0
linear_acceleration_X    0
linear_acceleration_Y    0
linear_acceleration_Z    0
dtype: int64

In [31]:
# Check number of missing values
len(X_test) - X_test.count()

row_id                   0
series_id                0
measurement_number       0
orientation_X            0
orientation_Y            0
orientation_Z            0
orientation_W            0
angular_velocity_X       0
angular_velocity_Y       0
angular_velocity_Z       0
linear_acceleration_X    0
linear_acceleration_Y    0
linear_acceleration_Z    0
dtype: int64

### Data preprocessing

In [32]:
# Get column names for X_train
col_names = X_train.columns
col_names

Index(['row_id', 'series_id', 'measurement_number', 'orientation_X',
       'orientation_Y', 'orientation_Z', 'orientation_W', 'angular_velocity_X',
       'angular_velocity_Y', 'angular_velocity_Z', 'linear_acceleration_X',
       'linear_acceleration_Y', 'linear_acceleration_Z'],
      dtype='object')

In [33]:
# Get column names for X_test
col_names2 = X_test.columns
col_names2

Index(['row_id', 'series_id', 'measurement_number', 'orientation_X',
       'orientation_Y', 'orientation_Z', 'orientation_W', 'angular_velocity_X',
       'angular_velocity_Y', 'angular_velocity_Z', 'linear_acceleration_X',
       'linear_acceleration_Y', 'linear_acceleration_Z'],
      dtype='object')

In [34]:
# Group X_train by series_id using the mean of the features and drop measurement_number
X_train_group = X_train[["series_id","orientation_X", "orientation_Y", 'orientation_Z', 'orientation_W', 'angular_velocity_X',
       'angular_velocity_Y', 'angular_velocity_Z', 'linear_acceleration_X',
       'linear_acceleration_Y', 'linear_acceleration_Z']].groupby(['series_id']).mean()
X_train_group

,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
series_id,,,,,,,,,,
0,-0.758666,-0.634008,-0.105474,-0.106470,-0.002481,-0.003306,0.007532,0.263418,2.984195,-9.320391
1,-0.958606,0.241867,0.031650,-0.146876,0.004605,-0.007757,0.006206,0.121867,2.768193,-9.388899
2,-0.512057,-0.846171,-0.129371,-0.071082,0.002646,-0.009232,0.027989,0.149711,2.886745,-9.395783
3,-0.939169,0.310140,0.038955,-0.142319,0.000624,-0.002804,0.007887,0.201791,2.657922,-9.451164
4,-0.891301,0.428144,0.060056,-0.136460,0.006969,0.046109,-0.142385,-0.086171,2.981498,-9.349988
...,...,...,...,...,...,...,...,...,...,...
3805,-0.228787,0.961730,0.144690,-0.042361,0.002519,0.008479,-0.021297,0.266687,2.745177,-9.465299
3806,0.542262,0.826590,0.129890,0.076335,-0.000453,0.003409,-0.000416,0.136476,2.900305,-9.367247
3807,-0.416511,0.895374,0.140185,-0.070016,0.031838,0.103777,-0.338278,-0.061158,3.660786,-9.192466


In [35]:
# Group X_test by series_id using the mean of the features and drop measurement_number
X_test_group = X_test[["series_id","orientation_X", "orientation_Y", 'orientation_Z', 'orientation_W', 'angular_velocity_X',
       'angular_velocity_Y', 'angular_velocity_Z', 'linear_acceleration_X',
       'linear_acceleration_Y', 'linear_acceleration_Z']].groupby(['series_id']).mean()
X_test_group

,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
series_id,,,,,,,,,,
0,0.919651,-0.362914,-0.047720,0.141749,-0.003581,0.083816,-0.254101,-0.151913,3.043463,-9.327918
1,0.043135,-0.987147,-0.147972,0.013861,0.019435,0.255122,-0.804857,0.160749,3.185698,-9.288633
2,-0.914825,0.376669,0.051761,-0.136139,0.002489,0.004208,0.010646,0.099235,2.959077,-9.395998
3,0.928206,0.338873,0.060741,0.138377,-0.010286,-0.170136,0.576255,0.584179,3.364273,-9.207547
4,-0.869592,0.470784,0.062041,-0.135351,-0.003275,-0.011757,0.022096,0.320250,2.683565,-9.338934
...,...,...,...,...,...,...,...,...,...,...
3811,-0.445217,-0.882418,-0.137777,-0.064218,-0.006429,0.029837,-0.080352,0.181259,2.930360,-9.411496
3812,-0.204659,0.967105,0.145481,-0.040604,-0.004471,-0.022723,0.063266,0.251856,2.924053,-9.344760
3813,0.474399,0.867587,0.133288,0.066814,-0.000403,-0.011272,0.054685,-0.026059,2.978274,-9.338603


In [0]:
X_test_group.reset_index(drop=False, inplace=True)

In [37]:
X_test_group.head()

,series_id,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0,0.919651,-0.362914,-0.047720,0.141749,-0.003581,0.083816,-0.254101,-0.151913,3.043463,-9.327918
1,1,0.043135,-0.987147,-0.147972,0.013861,0.019435,0.255122,-0.804857,0.160749,3.185698,-9.288633
2,2,-0.914825,0.376669,0.051761,-0.136139,0.002489,0.004208,0.010646,0.099235,2.959077,-9.395998
3,3,0.928206,0.338873,0.060741,0.138377,-0.010286,-0.170136,0.576255,0.584179,3.364273,-9.207547
4,4,-0.869592,0.470784,0.062041,-0.135351,-0.003275,-0.011757,0.022096,0.320250,2.683565,-9.338934


In [38]:
# Merge X_train_group and y_train
train_full = pd.merge(X_train_group, y_train, on='series_id', how='inner')
train_full

,series_id,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,group_id,surface
0,0,-0.758666,-0.634008,-0.105474,-0.106470,-0.002481,-0.003306,0.007532,0.263418,2.984195,-9.320391,13,fine_concrete
1,1,-0.958606,0.241867,0.031650,-0.146876,0.004605,-0.007757,0.006206,0.121867,2.768193,-9.388899,31,concrete
2,2,-0.512057,-0.846171,-0.129371,-0.071082,0.002646,-0.009232,0.027989,0.149711,2.886745,-9.395783,20,concrete
3,3,-0.939169,0.310140,0.038955,-0.142319,0.000624,-0.002804,0.007887,0.201791,2.657922,-9.451164,31,concrete
4,4,-0.891301,0.428144,0.060056,-0.136460,0.006969,0.046109,-0.142385,-0.086171,2.981498,-9.349988,22,soft_tiles
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3805,3805,-0.228787,0.961730,0.144690,-0.042361,0.002519,0.008479,-0.021297,0.266687,2.745177,-9.465299,55,tiled
3806,3806,0.542262,0.826590,0.129890,0.076335,-0.000453,0.003409,-0.000416,0.136476,2.900305,-9.367247,67,wood
3807,3807,-0.416511,0.895374,0.140185,-0.070016,0.031838,0.103777,-0.338278,-0.061158,3.660786,-9.192466,48,fine_concrete
3808,3808,-0.264584,0.952734,0.142060,-0.045705,0.001128,-0.029315,0.088592,0.218964,3.075475,-9.345442,54,tiled


In [39]:
# Check number of missing values
len(train_full) - train_full.count()

series_id                0
orientation_X            0
orientation_Y            0
orientation_Z            0
orientation_W            0
angular_velocity_X       0
angular_velocity_Y       0
angular_velocity_Z       0
linear_acceleration_X    0
linear_acceleration_Y    0
linear_acceleration_Z    0
group_id                 0
surface                  0
dtype: int64

In [0]:
#Save the 'Id' column
train_series_id = train_full['series_id']
test_series_id = X_test_group['series_id']
#Now drop the  'Id' colum since it's unnecessary for the prediction process.
train_full.drop(["series_id", "group_id"], axis = 1, inplace = True)
X_test_group.drop("series_id", axis = 1, inplace = True)

### Encoding

In [44]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
train_full["surface"] = label.fit_transform(train_full["surface"])
train_full

,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,surface
0,-0.758666,-0.634008,-0.105474,-0.106470,-0.002481,-0.003306,0.007532,0.263418,2.984195,-9.320391,2
1,-0.958606,0.241867,0.031650,-0.146876,0.004605,-0.007757,0.006206,0.121867,2.768193,-9.388899,1
2,-0.512057,-0.846171,-0.129371,-0.071082,0.002646,-0.009232,0.027989,0.149711,2.886745,-9.395783,1
3,-0.939169,0.310140,0.038955,-0.142319,0.000624,-0.002804,0.007887,0.201791,2.657922,-9.451164,1
4,-0.891301,0.428144,0.060056,-0.136460,0.006969,0.046109,-0.142385,-0.086171,2.981498,-9.349988,6
...,...,...,...,...,...,...,...,...,...,...,...
3805,-0.228787,0.961730,0.144690,-0.042361,0.002519,0.008479,-0.021297,0.266687,2.745177,-9.465299,7
3806,0.542262,0.826590,0.129890,0.076335,-0.000453,0.003409,-0.000416,0.136476,2.900305,-9.367247,8
3807,-0.416511,0.895374,0.140185,-0.070016,0.031838,0.103777,-0.338278,-0.061158,3.660786,-9.192466,2
3808,-0.264584,0.952734,0.142060,-0.045705,0.001128,-0.029315,0.088592,0.218964,3.075475,-9.345442,7


### Modeling

In [0]:
# Summarize the X and y variables.
X = train_full.drop("surface", axis=1)
y = train_full.surface

In [0]:
# split the dataset into training and validation datasets
from sklearn import model_selection
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.25, random_state=0,
                                                   stratify=y)

In [0]:
# Model building. Following function is used to train a model and to compute the accuracy score
def train_model(classifier, train_vector, train_labels, valid_vector, valid_labels, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(train_vector, train_labels)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(valid_vector)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_labels)

In [0]:
# Creating the models
models = [LogisticRegression(random_state= 1), SVC(random_state= 1), SGDClassifier(random_state= 1), GradientBoostingClassifier(random_state= 1), RandomForestClassifier(random_state= 1),
            XGBClassifier(random_state= 1), DecisionTreeClassifier(random_state= 1), BaggingClassifier(DecisionTreeClassifier(random_state=1)), LGBMClassifier(random_state= 1), AdaBoostClassifier(random_state= 1),
          KNeighborsClassifier()                                      
             ]
names = ['Logistic Regression','Support Vector Classifier','Stochastic Gradient Descent','Gradient Boosting Tree','Random Forest',
         'XGBoost', 'Decision Tree', 'Bagging Decision Tree', 'Light GBM', 'Adaboost', "KNN"
         ]

In [60]:
for model, name in zip(models,names):
  score = train_model(model, X_train, y_train, X_valid, y_valid)
  print(f"- {name}: Accuracy: {round(score,3)}")


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


- Logistic Regression: Accuracy: 0.318
- Support Vector Classifier: Accuracy: 0.272
- Stochastic Gradient Descent: Accuracy: 0.281
- Gradient Boosting Tree: Accuracy: 0.726
- Random Forest: Accuracy: 0.771
- XGBoost: Accuracy: 0.697
- Decision Tree: Accuracy: 0.651
- Bagging Decision Tree: Accuracy: 0.748
- Light GBM: Accuracy: 0.761
- Adaboost: Accuracy: 0.176
- KNN: Accuracy: 0.638


In [86]:
# LR
accuracy_LGBM = train_model(LGBMClassifier(num_leaves= 5, max_bin = 50, learning_rate = 0.1, random_state=1), X_train, y_train, X_valid, y_valid)
print ("LGBM: ", accuracy_LGBM)

LGBM:  0.6778593913955928


In [71]:
# Gridsearch for LGBM
from sklearn import model_selection
import math
param_grid = {'learning_rate': [0.1,0.5,0.3],
              'max_bin': [500, 1000, 10000]
              } 
 
clf = model_selection.GridSearchCV(LGBMClassifier(num_leaves= 1000), param_grid = param_grid, scoring = "accuracy", cv = 5)
clf.fit(X_train, y_train)
print("The best parameters are: ",clf.best_params_)
print("The best score achieved is: ",clf.best_score_)

The best parameters are:  {'learning_rate': 0.3, 'max_bin': 1000}
The best score achieved is:  0.7486908013177715


In [0]:
# Predictions on test set
model_lgb = LGBMClassifier(num_leaves= 5, max_bin = 50, learning_rate = 0.1, random_state=1)
model_lgb.fit(X_train, y_train)
y_lgb_enc = model_lgb.predict(X_test_group)
y_lgb = label.inverse_transform(y_lgb_enc)

In [90]:
# Save predictions in format used for competition scoring
output = pd.DataFrame({'series_id': test_series_id,
                       'surface': y_lgb})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
